### **Student Diet Example**
You want to save money eating while remaining healthy. A healthy diet requires at least P=6 units of protein, C=15 units of carbohydrates, F=5 units of fats and V=7 units of vitamins. Due to compounding factors (blizzard during Lent), our campus only has these options:


In [5]:
import pyomo.environ as pe
import pyomo.opt as po
import pandas as pd

In [6]:
df = pd.read_csv("https://ndcbe.github.io/CBE60499/data/student_diet.csv")
df.head()
m = pe.ConcreteModel()

**Sets**

In [7]:
nutrients = {'P', 'C', "F", "V"}
modes = {"takeaway", "vegtables", "bread"}

m.N = pe.Set(initialize=nutrients)
m.M = pe.Set(initialize=modes)

    (type: set).  This WILL potentially lead to nondeterministic behavior in
    Pyomo
    (type: set).  This WILL potentially lead to nondeterministic behavior in
    Pyomo


**Parameters**

In [8]:
price = {
    "takeaway": 5,
    "vegtables": 1,
    "bread": 2
}

m.price = pe.Param(m.M, initialize=price)

vals = {
    ('takeaway', 'P'): 3.0,
    ('takeaway', 'C'): 3.0,
    ('takeaway', 'F'): 2.0,
    ('takeaway', 'V'): 1.0,
    ('vegtables', 'P'): 1.0,
    ('vegtables', 'C'): 2.0,
    ('vegtables', 'F'): 0.0,
    ('vegtables', 'V'): 4.0,
    ('bread', 'P'): 0.5,
    ('bread', 'C'): 4.0,
    ('bread', 'F'): 1.0,
    ('bread', 'V'): 0.0
 }

m.vals = pe.Param(m.M, m.N, initialize=vals)

nr = {
    'P':6, 
    'C':15, 
    "F": 5, 
    "V": 7
}

m.nr = pe.Param(m.N, initialize=nr)

**Variables**

In [9]:
m.x = pe.Var(m.M, domain=pe.NonNegativeReals)

**Constraints**

In [10]:
def cons1(m, j):
    return sum(m.x[i]*m.vals[i, j] for i in m.M) >= m.nr[j]
m.cons = pe.Constraint(m.N, rule=cons1)


**Objective**

In [11]:
def objective(m):
    a =  sum(m.x[i]*m.price[i] for i in m.M)
    return a

m.TotCost = pe.Objective(rule=objective, sense=pe.minimize)

**Degree of Freedom Analysis**

We will later learn more about how to factor inequality constraints into degree of freedom analysis. For now, please count the number of equality and inequality constraints separately.


In [12]:
# Specify the solver
solver = po.SolverFactory('glpk')

# Solve
results = solver.solve(m)
for c in m.component_data_objects(pe.Var):
    print(c.name,"\t", pe.value(c))

x[bread] 	 3.0
x[vegtables] 	 1.5
x[takeaway] 	 1.0


**Take Away Messages**

Linear programs are convex. We will learn this means all local optima are global optima.
Nonlinear programs may be nonconvex. For nonconvex problems, there often existings many local optima that are not also global optima.